# Glycolysis Metabolic Network

This metabolic network consists of reactions involved in the glycolysis pathway. The default objective function is the maximization of ATP production (abbreviated as atp in the metabolic model). The figure below serves as a visual representation of the network.

![network](data/glycolysis_network.png)

## Loading Libraries and Importing the Model

We import the glycolysis model from the [BiGG Models](http://bigg.ucsd.edu) database using the cobrapy package. We also import the necessary packages for plotting and analysis.

In [1]:
# libraries
import cobra

# import the glycolysis model
model = cobra.io.read_sbml_model('data/glycolysis.xml')

## Exploration

The `model.objective` is already set to maximize ATP prodcution- We can explore the model by printing out the reactions outputs and metabolites in the model.

In [16]:
print("Reaction Outputs")
for row in model.reactions:
    # print(row.name, ":", row.id, ":", row.lower_bound, ":", row.upper_bound)
    print(str(row.name) + ", ", end = "")

print("\n\nMetabolites")
for row in model.metabolites:
    print(str(row) + ", ", end = "")

Reaction Outputs
GLCtr, GLCPTS, HEX, PYRtr, LACtr, Htr, O2tr, H2Otr, PGI, PFK, FBP, FBA, TPI, GAPD, PGK, PGM, ENO, PYK, LDH, BIOMASS, EX_glc_D, EX_h, EX_pyr, EX_lac_D, EX_o2, EX_h2o, 

Metabolites
h_e, glc_D, h, glc_D_e, pyr, pep, g6p, adp, atp, pyr_e, lac_D, lac_D_e, o2_e, o2, h2o_e, h2o, f6p, fdp, pi, g3p, dhap, nad, 13dpg, nadh, 3pg, 2pg, 

## Flux Balance Analysis
Once we have gathered the information about the reactions, we perform FBA to maximize ATP production. How many active reactions are there in the model?

In [18]:
# perform FBA
fluxes = model.optimize().fluxes
ar = fluxes[fluxes.round(3) > 0].index
print(f"There are {len(ar)} active reactions:", end = " ")
for r in ar:
    print(model.reactions.get_by_id(r).name + ", ", end = "")

There are 16 active reactions: GLCPTS, LACtr, PGI, PFK, FBA, TPI, GAPD, PGK, PGM, ENO, PYK, LDH, BIOMASS, EX_h, EX_lac_D, EX_h2o, 


Next we are interested in identifying the role of the reaction catalyzed by lactate dehydrogenase (LDH in the model), beyond its enzymatic function. We want to know whether it is an essential reaction and in which context it performs its enzymatic function

In [23]:
if __name__ == "__main__":
    import cobra
    from cobra import Model, Reaction, Metabolite
    from cobra.flux_analysis import single_reaction_deletion

    # extract the reaction id for LDH
    for i in model.reactions:
        if i.name == "LDH": ID = i.id
    deletion_results = single_reaction_deletion(model)
    # lower_bound controls the reverse reaction -> we set all but a to 0: nothing gets in but A

    mask = deletion_results["growth"].round(3) == 0
    if {ID} in list(deletion_results[mask]["ids"]):
        print("LDH is an essential reaction!")
    else:
        print("LDS is NOT an essential")

    print("\n", model.reactions.get_by_id(ID))


LDH is an essential reaction!

 R19: nadh + pyr <=> lac_D + nad


In [24]:
# summary of nadh produced by Lac_D
print(model.metabolites.nadh.summary())

nadh
====
Formula: None

Producing Reactions
-------------------
Percent Flux Reaction                          Definition
100.00%   20      R14 g3p + nad + pi <=> 13dpg + h + nadh

Consuming Reactions
-------------------
Percent Flux Reaction                 Definition
100.00%  -20      R19 nadh + pyr <=> lac_D + nad
